In [1]:
import pandas as pd
from IPython.display import display


In [2]:
'''
   run_bash_cmdning django script and jupyter in this case have
   different working directories
   so this is done to make sure that the data path is correct for both
'''
import os
cwd = os.getcwd()
curr_dir = cwd.split(os.path.sep)[-1]
IS_NOTEBOOK = curr_dir == 'scripts'

if IS_NOTEBOOK: 
   os.chdir('../')

NEW_CWD = os.getcwd()
DATA_PATH = os.path.join(NEW_CWD, 'data','all-states-history.csv')
print('cwd before checking: ', cwd)
print('DATA_PATH: ', DATA_PATH)
print('cwd now: ', NEW_CWD)
print('IS_NOTEBOOK: ', IS_NOTEBOOK)

cwd before checking:  f:\Projects\geo-covid-backend\scripts
DATA_PATH:  f:\Projects\geo-covid-backend\data\all-states-history.csv
cwd now:  f:\Projects\geo-covid-backend
IS_NOTEBOOK:  True


In [3]:
''' Activate django env if is in notebook '''

if IS_NOTEBOOK:
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'geo_covid.settings')
    import django
    django.setup()

Db collection list:  ['django_session', 'auth_group_permissions', 'auth_permission', 'us_covid_api_polygon', 'auth_group', 'auth_user_user_permissions', 'django_admin_log', 'auth_user', 'django_migrations', 'us_covid_api_report', 'auth_user_groups', 'us_covid_api_state', 'django_content_type', 'us_covid_api_globalreport', '__schema__']
client:  {'host': 'localhost'}


In [4]:
'''
    Meta functions
'''
import subprocess

def decode_output(str):
    try:
        return str.decode('utf-8')
    except UnicodeDecodeError as e:
        return str.decode('latin-1')
def run_bash_cmd(command: str):
    print("Exec: ", command)
    process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    if output: print('Out: ', decode_output(output))
    if error: print('Err: ', decode_output(error))

In [5]:
'''
    Helper functions
'''
import shutil
import re

CASE_CONVERT_PATTERN = re.compile(r'(?<!^)(?=[A-Z])')
def camel_case_to_snake_case(string):
    return CASE_CONVERT_PATTERN.sub('_', string).lower()

def clear_migrations():
    try:
        shutil.rmtree("./us_covid_api/migrations/")
    except BaseException as e:
        print('Fail to clear migration: ',e)

def clean_cache():
    run_bash_cmd('python manage.py clear_cache')
    run_bash_cmd('python manage.py clean_pyc')
    run_bash_cmd(f'pyclean {NEW_CWD}')

def drop_database():
    from pymongo import MongoClient
    client = MongoClient('localhost', 27017)
    client.drop_database('geo-covid')

def clean_and_remigrate_db():
    run_bash_cmd("python manage.py flush --noinput")
    run_bash_cmd("python manage.py makemigrations us_covid_api")
    run_bash_cmd("python manage.py migrate")

def generate_drf_spec_schema():
    # generate schema for drf_spectacular
    run_bash_cmd("python manage.py spectacular --file schema.yml")

def print_settings():
    run_bash_cmd("python manage.py print_settings")

In [6]:
'''
    Important functions relating to script main functionality
'''
from tkinter import N
import numpy as np
from math import isnan
from us_covid_api.models import Report, State, Polygon, GlobalReport
from django.core.exceptions import ObjectDoesNotExist
from django.utils.timezone import make_aware

def parse_state_data(df_geo: pd.DataFrame):

    def parse_geo_data(record):
        return pd.Series([*record.fields.values(), record.fields['st_asgeojson']])
    def parse_geo_data_columns(geo_data):
        columns = list(geo_data.iloc[0].fields.keys()) + ['geometry']
        columns = list(map(lambda x: x.lower(), columns)) 
        return columns

    columns = parse_geo_data_columns(df_geo)
    final_column = ['name', 'geometry', 'state', 'stusab']
    temp = df_geo.apply(parse_geo_data, axis=1)
    temp.columns = columns

    df_geo = temp[final_column]
    df_geo.columns = final_column

    return df_geo

def load_states(state_df: pd.DataFrame):
    def generate_state(state_record):
        state = State(
            name=state_record['name'], 
            initials=state_record['stusab'], 
            id=state_record['state'],
        )
        state.save()
        polygon = Polygon(
            coordinates=state_record['geometry']['coordinates'],
            type=state_record['geometry']['type'],
            state_name=state_record['name'], 
            state_initials=state_record['stusab'], 
            state_id=state_record['state'],
        )
        polygon.save()
    
    state_df.apply(generate_state, axis = 1)
    print(f'State count: ', State.objects.count())
    print(f'Polygon count: ', Polygon.objects.count())

def load_reports(report_df: pd.DataFrame):
    def generate_report(report_record: pd.Series):
        try:
            state = State.objects.get(initials=report_record['state'])
        except ObjectDoesNotExist as e:
            print(f'Not found state: {report_record["state"]}')
            state = None
        if state is None: return None
        report_record.loc['state'] = state
        report_record.loc['date'] = make_aware(report_record.loc['date'])
        report_dict = {k:(v if not type(v) is float or not isnan(v) else None) for k, v in report_record.to_dict().items()}

        report_obj = Report(id=None, **report_dict)
        report_obj.save()

    report_df.apply(generate_report, axis=1)
    print(f'Report count: ', Report.objects.count())

def load_global_data(global_df: pd.DataFrame):
    def generate_global_report(report_record: pd.Series):
        report_record.loc['date'] = make_aware(report_record.loc['date'])
        report_dict = {k:(v if not type(v) is float or not isnan(v) else None) for k, v in report_record.to_dict().items()}

        report_obj = GlobalReport(id=None, **report_dict)
        report_obj.save()
    
    global_df.apply(generate_global_report, axis=1)
    print(f'Global Report count: ', GlobalReport.objects.count())

In [7]:
''' Testing code '''

def sample_code():
    df = pd.read_csv('data/all-states-history.csv', parse_dates=['date'])
    df.columns = list(df.columns.map(lambda x: camel_case_to_snake_case(x)))
    record = df.loc[0].copy()
    state = State(name='random', initials=record['state'])
    state.save()
    record.loc['state'] = state
    # record.pop('state')
    value_dict = record.to_dict()
    newdict = {k:(v if not type(v) is float or not isnan(v) else None) for k, v in value_dict.items()}
    print(newdict)
    obj = Report(id=None, **newdict)
    obj.save()
    print('DONE')

In [8]:
'''
    NOTE: this is the main function & entry point of the whole script (most important)
'''

from ast import arg


def run(*args):
    print('script args: ', args)
    # Default behavior is run all below tasks
    if 'no-clear-migrations' not in args: clear_migrations()
    if 'no-clean-cache' not in args: clean_cache()
    if 'no-drop-db' not in args: drop_database()
    if 'no-remigrate-db' not in args: clean_and_remigrate_db()
    if 'no-generate-schema' not in args: generate_drf_spec_schema()
    if 'no-print-settings' not in args: print_settings()
    print('-----DONE PREP-----------------')
    # Process global data (data for all states)
    df_global = pd.read_csv('data/national-history.csv', parse_dates=['date'])
    df_global.columns = list(df_global.columns.map(lambda x: camel_case_to_snake_case(x)))
    print('Global data: ', df_global.shape)
    load_global_data(df_global)
    # Local data processing step
    df = pd.read_csv('data/all-states-history.csv', parse_dates=['date'])
    df_geo = pd.read_json('data/us-state-boundaries.json')
    print('Shape df: ', df.shape)
    df.columns = list(df.columns.map(lambda x: camel_case_to_snake_case(x)))
    df_state = parse_state_data(df_geo)
    print('Shape df_state: ', df_state.shape)
    load_states(df_state)
    load_reports(df)

    print('Done importing')

In [9]:
# ''' Notebook test run '''
import threading
import time
NOTEBOOK_ARGS = [
]

if IS_NOTEBOOK:
    print("Main    : before creating thread")
    x = threading.Thread(target=run, args = NOTEBOOK_ARGS)
    print("Main    : before running thread")
    x.start()
    x.join()
    print("Main    : wait for the thread to finish")
    print("Main    : all done")

Main    : before creating thread
Main    : before running thread
script args:  ()
Exec:  python manage.py clear_cache
Out:  Db collection list:  ['django_session', 'auth_group_permissions', 'auth_permission', 'us_covid_api_polygon', 'auth_group', 'auth_user_user_permissions', 'django_admin_log', 'auth_user', 'django_migrations', 'us_covid_api_report', 'auth_user_groups', 'us_covid_api_state', 'django_content_type', 'us_covid_api_globalreport', '__schema__']
client:  {'host': 'localhost'}
Cache "default" has been cleared!

Exec:  python manage.py clean_pyc
Out:  Db collection list:  ['django_session', 'auth_group_permissions', 'auth_permission', 'us_covid_api_polygon', 'auth_group', 'auth_user_user_permissions', 'django_admin_log', 'auth_user', 'django_migrations', 'us_covid_api_report', 'auth_user_groups', 'us_covid_api_state', 'django_content_type', 'us_covid_api_globalreport', '__schema__']
client:  {'host': 'localhost'}

Exec:  pyclean f:\Projects\geo-covid-backend
Exec:  python man

In [10]:
'''
    Testing queries
'''

import timeit
from random import randrange, seed, randint
from datetime import timedelta
import datetime
import threading
import time
# Settings
DAYS_INTERVAL = 5 
seed(0)


# Helper
def random_date(start=datetime.date(2020, 1, 13), end=datetime.date(2021, 3, 7)):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

def random_state_initials():
    states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
    return states[randint(0, len(states) - 1)]

def examine_thread_speed(function, args = (), verbose = True):
    if verbose: print('--------------------------')
    # NOTE: there is threading overhead involved
    if args:
        x = threading.Thread(target=function, args = (*args, verbose,))
    else:
        new_func = lambda: function(verbose=verbose)
        x = threading.Thread(target=new_func)
    if verbose: print("Before running thread: ", function.__name__, ", arg: ", args)
    start = timeit.default_timer()
    x.start()
    x.join()
    stop = timeit.default_timer()
    time_diff = stop - start
    if verbose: print('Done, Execution Time (second): ', time_diff)
    return time_diff  


# Tests
def stats_one_day_all_states(date, verbose = True):
    if verbose: print('Date: ', date)
    time_range = (make_aware(datetime.datetime.combine(date, datetime.time.min)),
                make_aware(datetime.datetime.combine(date, datetime.time.max)))
    reports = Report.objects.filter(date__range= time_range)
    if verbose: print('Report retrieved: ', reports.count())
    if len(reports) == 0: print('No reports found at given date!')

def stats_day_range_all_states(start_day, verbose = True):
    end_day = start_day + datetime.timedelta(days=DAYS_INTERVAL)
    if verbose: print('Start day: ', start_day, ', end day: ', end_day)
    time_range = (make_aware(datetime.datetime.combine(start_day, datetime.time.min)),
                make_aware(datetime.datetime.combine(end_day, datetime.time.max)))
    reports = Report.objects.filter(date__range= time_range)
    if verbose: print('Report retrieved: ', reports.count())
    if len(reports) == 0: print('No reports found at given date!')
            
def stats_one_state_all_days(state_initials, verbose = True):
    if verbose: print('State: ', state_initials)
    state = State.objects.filter(initials=state_initials).first()
    reports = Report.objects.filter(state_id= state.id)
    if verbose: print('Report retrieved: ', reports.count())
    if len(reports) == 0: print('No reports found at given date!')
# def stats_few_state_all_days(): pass # ! OUT OF SCOPE


# Results
if IS_NOTEBOOK:
    examine_thread_speed(stats_one_day_all_states, args=(random_date(),));

    start_day = random_date()
    examine_thread_speed(stats_day_range_all_states, args=(start_day,));

    examine_thread_speed(stats_one_state_all_days, args=(random_state_initials(),));

--------------------------
Before running thread:  stats_one_day_all_states , arg:  (datetime.date(2020, 11, 7),)
Date:  2020-11-07
Report retrieved:  56
Done, Execution Time (second):  0.08404030000002649
--------------------------
Before running thread:  stats_day_range_all_states , arg:  (datetime.date(2020, 8, 1),)
Start day:  2020-08-01 , end day:  2020-08-06
Report retrieved:  336
Done, Execution Time (second):  0.2963641999999709
--------------------------
Before running thread:  stats_one_state_all_days , arg:  ('NJ',)
State:  NJ
Report retrieved:  392
Done, Execution Time (second):  0.39982480000003306


In [11]:

# Helper
def benchmark(function, args_list, inner_verbose=False):
    print('-----------------------------')
    results = []
    for args in args_list:
        time_delta = examine_thread_speed(function, args, verbose=inner_verbose)
        results.append(time_delta)
    print('Run results (second): ', results)
    print('Stats: ')
    display(pd.DataFrame(results).describe())

# Settings
REPEATS = 20
dates = [random_date() for _ in range(REPEATS)]
states_initials_list = [random_state_initials() for _ in range(REPEATS)]

# Results
to_args = lambda list_val: [(x,) for x in list_val]
if IS_NOTEBOOK:
    benchmark(stats_one_day_all_states, to_args(dates))
    benchmark(stats_day_range_all_states, to_args(dates))
    benchmark(stats_one_state_all_days, to_args(states_initials_list))

-----------------------------
Run results (second):  [0.14403029999999717, 0.2008822999999893, 0.19212199999998347, 0.1317325999999639, 0.0822992999999883, 0.1141411000000403, 0.145867399999986, 0.12610499999999547, 0.18371250000001282, 0.14979120000003832, 0.0980623000000378, 0.07858390000001236, 0.10971990000001597, 0.13483509999997523, 0.1354462999999555, 0.10362909999997783, 0.0895345000000134, 0.09287520000003724, 0.07431580000002214, 0.03387550000002193]
Stats: 


,0
count,20.000000
mean,0.121078
std,0.042141
min,0.033876
25%,0.092040
50%,0.120123
75%,0.144490
max,0.200882


-----------------------------
Run results (second):  [0.3626403000000096, 0.25696879999998146, 0.2751777999999945, 0.2738139999999589, 0.31733729999996285, 0.28675090000001546, 0.27003669999999147, 0.2610806999999795, 0.27021609999997054, 0.26638080000003583, 0.2761893999999643, 0.3380300000000034, 0.2646847000000321, 0.2698227999999858, 0.29372919999997293, 0.3137947999999824, 0.2801739000000225, 0.2702910000000429, 0.2638862999999674, 0.12706470000000536]
Stats: 


,0
count,20.000000
mean,0.276904
std,0.044840
min,0.127065
25%,0.265957
50%,0.272053
75%,0.288495
max,0.362640


-----------------------------
Run results (second):  [0.364482799999962, 0.29270379999996976, 0.28990459999999985, 0.40952929999997423, 0.3053614999999468, 0.3197460000000092, 0.3065470000000232, 0.2561132000000157, 0.2663395999999807, 0.25570170000003145, 0.2560621000000083, 0.2541390000000092, 0.2948746000000142, 0.2637813000000051, 0.33871269999997367, 0.27711250000004384, 0.2571975999999836, 0.2638582999999812, 0.2878420000000119, 0.3173375000000078]
Stats: 


,0
count,20.000000
mean,0.293867
std,0.041053
min,0.254139
25%,0.262135
50%,0.288873
75%,0.309245
max,0.409529


In [12]:
if IS_NOTEBOOK:
    run_bash_cmd('jupyter nbconvert --to python .\scripts\load_csv_to_database.ipynb')

Exec:  jupyter nbconvert --to python .\scripts\load_csv_to_database.ipynb
